<a href="https://colab.research.google.com/github/skynunu/2021_cau_oss_hackathon/blob/main/hackathon(%EA%B0%80%EA%B3%84%EC%A0%95)_team20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2021_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 두개의 테스트 데이터 셋 분류 정확도에 대한 weighted sum
 *  model.evaluate(x_test1, y_test1) + model.evaluate(x_test2, y_test2) * 2
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예: 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성
 *  단, tensorflow.data 및 dataset API를 사용하기 위해 변경하는 것은 허용. 이 경우, model evaluation 파트도 해당 API를 쓰도록 변경  
*  주석이 소스코드와 맞지 않거나 미비할 경우






SyntaxError: ignored

# **1. 초기 환경 설정**



In [ ]:
# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from keras.utils import np_utils
import numpy as np

# 데이터셋 로드 (Training dataset: CIFAR10, test dataset: CIFAR10 & CIFAR10의 변형)
(x_train, y_train), (x_test1, y_test1) = keras.datasets.cifar10.load_data()

test_ds2 = tfds.load('cifar10_1/v6', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.as_numpy(test_ds2)
x_test2, y_test2 = test_ds2['image'], test_ds2['label']

#분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test1 = np_utils.to_categorical(y_test1)
y_test2 = np_utils.to_categorical(y_test2)

# 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

# **2. 데이터 전처리**



In [8]:
# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test1_after, x_test2_after를 변경하지 말 것
x_train_after = x_train / 255.0
x_test1_after = x_test1 / 255.0
x_test2_after = x_test2 / 255.0

x_train_after= x_train_after.astype('float32')
x_test1_after= x_test1_after.astype('float32')
x_test2_after= x_test2_after.astype('float32')

In [9]:
# train dataset data augmentation
batch_size =64
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True )

train_dataset = datagen.flow(x_train_after, y_train, batch_size=batch_size)

In [10]:
# batch 사이즈 설정 test1_after, test2_after
BATCH_SIZE =64

test1_dataset = tf.data.Dataset.from_tensor_slices((x_test1_after, y_test1))
test2_dataset = tf.data.Dataset.from_tensor_slices((x_test2_after, y_test2))

test1_dataset = test1_dataset.batch(BATCH_SIZE)
test2_dataset = test2_dataset.batch(BATCH_SIZE)


# **3. 모델 생성**

In [22]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, GlobalAveragePooling2D,Activation, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.callbacks import ModelCheckpoint

In [23]:
     #vgg16 내부구조 참고
     #vgg16에 없는 배치정규화를 추가하여 층을 구성하였고 vgg16의 conv층의 마지막 3-layers인 conv3-512를 제외
     #fine tuning의 경우  GlobalAveragePooling2D(), Dropout(), Dense(512), Dense(256), BatchNormalization를 섞어서 구성
        
     model = Sequential([ 
        Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),                 
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(128, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),

        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(512, (3, 3), kernel_initializer='he_uniform', padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D((2, 2)),
        
        #Flatten(),
        GlobalAveragePooling2D(),
        Dropout(0.5),
         
        Dense(512),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        
        Dense(256),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),    

        Dense(10, activation='softmax'),

        
     ])
 

In [24]:
# model compile
from keras import optimizers
#'adam'
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
# model 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

In [26]:
# model early point 설정
from keras.callbacks import EarlyStopping
earlystopping = EarlyStopping
earlystopping = EarlyStopping(monitor='val_loss',  
                              patience=12,         
                             )
# 모니터 기준 설정 (val loss) 
# 12회 Epoch동안 개선되지 않는다면 종료 (patience)

In [27]:
#model fit
model.fit(train_dataset, epochs = 200,  callbacks=[cp_callback, earlystopping], validation_data=(test2_dataset))

Epoch 1/200
307/782 [==========>...................] - ETA: 41s - loss: 1.9806 - accuracy: 0.2234

KeyboardInterrupt: ignored

# **4. 모델 저장**

In [19]:
save_path = '/content/'
team_name = 'team20_transferlearning'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**


In [20]:
save_path = '/content/'
team_name = 'team20_transferlearning'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

print("test1 dataset 정확도",model.evaluate(x_test1_after, y_test1))
print("test2 dataset 정확도",model.evaluate(x_test2_after, y_test2))

63/63 [==============================] - 1s 16ms/step - loss: 0.8430 - accuracy: 0.7725


[0.842998743057251, 0.7724999785423279]